In [16]:
from IPython.display import display, Markdown, Latex
import pandas as pd
import numpy as np
import json
import os

## Experiments and Functions

In [42]:
experiments = [
            {'id': '1', 'source':'imdb', 'target':'uwcse', 'predicate':'workedunder', 'to_predicate':'advisedby', 'arity': 2},
            #{'id': '2', 'source':'uwcse', 'target':'imdb', 'predicate':'advisedby', 'to_predicate':'workedunder', 'arity': 2},
            {'id': '3', 'source':'imdb', 'target':'cora', 'predicate':'workedunder', 'to_predicate':'samevenue', 'arity': 2},
            #{'id': '4', 'source':'cora', 'target':'imdb', 'predicate':'samevenue', 'to_predicate':'workedunder', 'arity': 2},
            ##{'id': '5', 'source':'cora', 'target':'imdb', 'predicate':'sametitle', 'to_predicate':'workedunder', 'arity': 2},
            ##{'id': '6', 'source':'imdb', 'target':'cora', 'predicate':'workedunder', 'to_predicate':'sametitle', 'arity': 2},
            ##{'id': '5', 'source':'uwcse', 'target':'cora', 'predicate':'advisedby', 'to_predicate':'samevenue', 'arity': 2},
            ##{'id': '6', 'source':'cora', 'target':'uwcse', 'predicate':'samevenue', 'to_predicate':'advisedby', 'arity': 2},
            #{'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2},
            #{'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2},
            {'id': '9', 'source':'nell_sports', 'target':'nell_finances', 'predicate':'teamplayssport', 'to_predicate':'companyeconomicsector', 'arity': 2},
            {'id': '10', 'source':'nell_finances', 'target':'nell_sports', 'predicate':'companyeconomicsector', 'to_predicate':'teamplayssport', 'arity': 2},
]

In [21]:
def load_data(experiments, metric, directory):
    data = {}
    for item in experiments:
        experiment_title = item['id'] + '_' + item['source'] + '_' + item['target']
        
        if(metric != ''):
            file_path = os.getcwd() + directory + experiment_title + '/' + experiment_title + '_fasttext_{}.json'.format(metric)
        else:
            file_path = os.getcwd() + directory + experiment_title + '/' + experiment_title + '.json'
            
        #if(os.path.isfile(file_path)):
        with open(file_path, 'r') as fp:
            results = json.load(fp)

            data[experiment_title] = results
    return data

## Estudo do Uso de Stopwords

### Depth-First Mapping

In [13]:
# Setting paths parameters
mapping_order = 'deep'
k = 1

#Removing Yeast and Twitter pair of experiments because there is no stopwords in predicates
exp = experiments.copy()
exp.remove({'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2})
exp.remove({'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2})

In [39]:
# Using stopwords
path = f'/results/transfer/{mapping_order}/w_stopwords/k_{k}/'
data_softcosine_stopwords = load_data(experiments, 'softcosine', path)
data_euclidean_stopwords = load_data(experiments, 'euclidean', path)
data_wmd_stopwords = load_data(experiments, 'wmd', path)
data_rwmd_stopwords = load_data(experiments, 'relax-wmd', path)

# Removing stopwords
path = f'/results/transfer/{mapping_order}/no_stopwords/k_{k}/'
data_softcosine_no_stopwords = load_data(experiments, 'softcosine', path)
data_euclidean_no_stopwords = load_data(experiments, 'euclidean', path)
data_wmd_no_stopwords = load_data(experiments, 'wmd', path)
data_rwmd_no_stopwords = load_data(experiments, 'relax-wmd', path)

baseline_path = f'/results/baselines/'
baseline_data = load_data(experiments, '', baseline_path)

for metric in ['AUC ROC', 'AUC PR', 'CLL', 'Learning time']: #, 'Recall', 'F1', 'Precision', 'Learning and Revision time', 'Inference time']:
    display(Markdown('# Results for ' + metric))
    
    for j in range(len(experiments)):
            
        table = []
        dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
        if dataset in data_euclidean_stopwords:
            mapping_time = ''
            results_using_stopwords = {}
            results_removing_stopwords = {}

            # Results using Stopwords
            results_using_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
            results_using_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
            results_using_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
            results_using_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])

            # Results removing stopwords
            results_removing_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_no_stopwords[str(dataset)]])
            results_removing_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_no_stopwords[str(dataset)]])
            results_removing_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_no_stopwords[str(dataset)]])
            results_removing_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_no_stopwords[str(dataset)]])
            
            if(experiments[j]['id'] != '10'):
                treeboostler = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in baseline_data[str(dataset)]])
                #treeboostler_no_revision = np.array([(np.array([item['transfer_{}'.format(amount)]['parameter'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #print(treeboostler)
            rdnb = np.array([(np.array([item['rdn_b'][metric] for item in sublist])).mean() for sublist in baseline_data[str(dataset)]])
            
            if(experiments[j]['id'] != '10'):
                table.append([dataset,
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
                '%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                             ])
            else:
                table.append([dataset,
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
                #'%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                             ])
            
            table.append(['', '', '', '', '', '', ''])
           
            if(experiments[j]['id'] != '10'):
                table.append([dataset + '*',
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                '%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                             ])
            else:
                table.append([dataset + '*',
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                #'%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                             ])

        #display(pd.DataFrame(table, columns=['Experiment', 'RDN-B'])) #, 'Transfer Learning with Revision Theory - Relax WMD'])) #, 'Learning from scratch (RDN-B)']))
        display(pd.DataFrame(table, columns=['Experiment', 'Transfer Learning - SoftCosine', 'Transfer Learning - Euclidean', 'Transfer Learning - WMD', 'Transfer Learning - Relax-WMD', 'TreeBoostler', 'RDN-B'])) #, 'Learning from scratch (RDN-B)']))
        #display(pd.DataFrame(table, columns=['Experiment', 'Transfer Learning', 'Transfer Learning with Revision Theory']))

# Results for AUC ROC

,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,1_imdb_uwcse,0.937 +/- 0.002,0.947 +/- 0.001,0.937 +/- 0.003,0.939 +/- 0.004,0.939 +/- 0.005,0.936 +/- 0.006
1,,,,,,,
2,1_imdb_uwcse*,0.937 +/- 0.004,0.937 +/- 0.007,0.937 +/- 0.005,0.937 +/- 0.004,0.939 +/- 0.005,0.936 +/- 0.006


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,3_imdb_cora,0.591 +/- 0.055,0.595 +/- 0.054,0.592 +/- 0.056,0.611 +/- 0.005,0.576 +/- 0.048,0.566 +/- 0.041
1,,,,,,,
2,3_imdb_cora*,0.593 +/- 0.057,0.600 +/- 0.048,0.600 +/- 0.048,0.592 +/- 0.058,0.576 +/- 0.048,0.566 +/- 0.041


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,4_cora_imdb,1.000 +/- 0.000,1.000 +/- 0.000,1.000 +/- 0.000,1.000 +/- 0.000,1.000 +/- 0.000,1.000 +/- 0.000
1,,,,,,,
2,4_cora_imdb*,1.000 +/- 0.000,1.000 +/- 0.000,1.000 +/- 0.000,1.000 +/- 0.000,1.000 +/- 0.000,1.000 +/- 0.000


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,9_nell_sports_nell_finances,0.731 +/- 0.017,0.733 +/- 0.058,0.737 +/- 0.089,0.739 +/- 0.038,0.746 +/- 0.019,0.762 +/- 0.041
1,,,,,,,
2,9_nell_sports_nell_finances*,0.712 +/- 0.010,0.739 +/- 0.057,0.717 +/- 0.022,0.720 +/- 0.103,0.746 +/- 0.019,0.762 +/- 0.041


...

,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,1_imdb_uwcse,12.386 +/- 1.135,13.195 +/- 3.957,13.831 +/- 0.970,37.649 +/- 0.748,15.725 +/- 12.166,10.457 +/- 1.078
1,,,,,,,
2,1_imdb_uwcse*,12.869 +/- 0.952,15.923 +/- 4.436,14.743 +/- 1.438,24.578 +/- 15.164,15.725 +/- 12.166,10.457 +/- 1.078


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,3_imdb_cora,110.966 +/- 8.470,112.129 +/- 18.092,112.681 +/- 11.092,137.193 +/- 16.142,247.337 +/- 126.544,133.098 +/- 12.012
1,,,,,,,
2,3_imdb_cora*,111.999 +/- 17.325,113.199 +/- 16.022,114.166 +/- 14.631,122.998 +/- 19.787,247.337 +/- 126.544,133.098 +/- 12.012


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,4_cora_imdb,10.256 +/- 0.152,10.067 +/- 0.134,10.027 +/- 0.171,36.222 +/- 0.151,10.998 +/- 0.080,3.830 +/- 0.148
1,,,,,,,
2,4_cora_imdb*,10.282 +/- 0.147,10.019 +/- 0.195,9.978 +/- 0.135,39.053 +/- 0.116,10.998 +/- 0.080,3.830 +/- 0.148


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,9_nell_sports_nell_finances,159.339 +/- 55.864,127.456 +/- 7.967,153.813 +/- 25.551,214.023 +/- 51.197,278.228 +/- 97.144,42.075 +/- 9.081
1,,,,,,,
2,9_nell_sports_nell_finances*,157.709 +/- 15.851,146.661 +/- 36.490,146.675 +/- 18.618,186.500 +/- 53.218,278.228 +/- 97.144,42.075 +/- 9.081


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,10_nell_finances_nell_sports,590.475 +/- 133.093,640.814 +/- 252.994,496.680 +/- 166.363,486.343 +/- 136.785,349.809 +/- 11.872,None
1,,,,,,,
2,10_nell_finances_nell_sports*,552.248 +/- 13.962,561.954 +/- 10.387,791.257 +/- 398.673,616.353 +/- 168.749,349.809 +/- 11.872,None


### Ranked-First Mapping

In [40]:
# Setting paths parameters
mapping_order = 'most_similar'
k = 1

In [43]:
# Using stopwords
path = f'/results/transfer/{mapping_order}/w_stopwords/k_{k}/'
data_softcosine_stopwords = load_data(experiments, 'softcosine', path)
data_euclidean_stopwords = load_data(experiments, 'euclidean', path)
data_wmd_stopwords = load_data(experiments, 'wmd', path)
data_rwmd_stopwords = load_data(experiments, 'relax-wmd', path)

# Removing stopwords
path = f'/results/transfer/{mapping_order}/no_stopwords/k_{k}/'
data_softcosine_no_stopwords = load_data(experiments, 'softcosine', path)
data_euclidean_no_stopwords = load_data(experiments, 'euclidean', path)
data_wmd_no_stopwords = load_data(experiments, 'wmd', path)
data_rwmd_no_stopwords = load_data(experiments, 'relax-wmd', path)

baseline_path = f'/results/baselines/'
baseline_data = load_data(experiments, '', baseline_path)

for metric in ['AUC ROC', 'AUC PR', 'CLL', 'Learning time']: #, 'Recall', 'F1', 'Precision', 'Learning and Revision time', 'Inference time']:
    display(Markdown('# Results for ' + metric))
    
    for j in range(len(experiments)):
            
        table = []
        dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
        if dataset in data_euclidean_stopwords:
            mapping_time = ''
            results_using_stopwords = {}
            results_removing_stopwords = {}

            # Results using Stopwords
            results_using_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
            results_using_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
            results_using_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
            results_using_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])

            # Results removing stopwords
            results_removing_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_no_stopwords[str(dataset)]])
            results_removing_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_no_stopwords[str(dataset)]])
            results_removing_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_no_stopwords[str(dataset)]])
            results_removing_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_no_stopwords[str(dataset)]])
            
            if(experiments[j]['id'] != '10'):
                treeboostler = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in baseline_data[str(dataset)]])
                #treeboostler_no_revision = np.array([(np.array([item['transfer_{}'.format(amount)]['parameter'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #print(treeboostler)
            rdnb = np.array([(np.array([item['rdn_b'][metric] for item in sublist])).mean() for sublist in baseline_data[str(dataset)]])
            
            if(experiments[j]['id'] != '10'):
                table.append([dataset,
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
                '%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                             ])
            else:
                table.append([dataset,
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
                #'%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                             ])
            
            table.append(['', '', '', '', '', '', ''])
           
            if(experiments[j]['id'] != '10'):
                table.append([dataset + '*',
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                '%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                             ])
            else:
                table.append([dataset + '*',
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                #'%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                             ])

        #display(pd.DataFrame(table, columns=['Experiment', 'RDN-B'])) #, 'Transfer Learning with Revision Theory - Relax WMD'])) #, 'Learning from scratch (RDN-B)']))
        display(pd.DataFrame(table, columns=['Experiment', 'Transfer Learning - SoftCosine', 'Transfer Learning - Euclidean', 'Transfer Learning - WMD', 'Transfer Learning - Relax-WMD', 'TreeBoostler', 'RDN-B'])) #, 'Learning from scratch (RDN-B)']))
        #display(pd.DataFrame(table, columns=['Experiment', 'Transfer Learning', 'Transfer Learning with Revision Theory']))

# Results for AUC ROC

,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,1_imdb_uwcse,0.938 +/- 0.007,0.946 +/- 0.007,0.938 +/- 0.003,0.936 +/- 0.006,0.939 +/- 0.005,0.936 +/- 0.006
1,,,,,,,
2,1_imdb_uwcse*,0.936 +/- 0.005,0.938 +/- 0.003,0.937 +/- 0.003,0.937 +/- 0.007,0.939 +/- 0.005,0.936 +/- 0.006


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,3_imdb_cora,0.576 +/- 0.057,0.611 +/- 0.003,0.601 +/- 0.046,0.612 +/- 0.004,0.576 +/- 0.048,0.566 +/- 0.041
1,,,,,,,
2,3_imdb_cora*,0.611 +/- 0.002,0.612 +/- 0.005,0.599 +/- 0.046,0.576 +/- 0.056,0.576 +/- 0.048,0.566 +/- 0.041


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,9_nell_sports_nell_finances,0.726 +/- 0.058,0.728 +/- 0.067,0.951 +/- 0.005,0.953 +/- 0.008,0.746 +/- 0.019,0.762 +/- 0.041
1,,,,,,,
2,9_nell_sports_nell_finances*,0.734 +/- 0.008,0.602 +/- 0.046,0.951 +/- 0.016,0.949 +/- 0.007,0.746 +/- 0.019,0.762 +/- 0.041


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,10_nell_finances_nell_sports,0.992 +/- 0.002,0.993 +/- 0.001,0.992 +/- 0.001,0.992 +/- 0.002,0.994 +/- 0.001,None
1,,,,,,,
2,10_nell_finances_nell_sports*,0.991 +/- 0.004,0.991 +/- 0.001,0.991 +/- 0.001,0.984 +/- 0.007,0.994 +/- 0.001,None


...

# Results for Learning time

,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,1_imdb_uwcse,12.572 +/- 0.597,13.451 +/- 2.704,13.936 +/- 1.235,20.811 +/- 0.970,15.725 +/- 12.166,10.457 +/- 1.078
1,,,,,,,
2,1_imdb_uwcse*,12.266 +/- 0.403,14.837 +/- 3.225,14.998 +/- 1.112,20.034 +/- 0.900,15.725 +/- 12.166,10.457 +/- 1.078


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,3_imdb_cora,105.714 +/- 8.626,117.315 +/- 7.685,113.748 +/- 6.394,122.018 +/- 8.152,247.337 +/- 126.544,133.098 +/- 12.012
1,,,,,,,
2,3_imdb_cora*,114.616 +/- 2.846,114.566 +/- 12.596,112.982 +/- 13.778,115.593 +/- 13.697,247.337 +/- 126.544,133.098 +/- 12.012


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,9_nell_sports_nell_finances,162.631 +/- 25.626,162.346 +/- 29.776,201.542 +/- 123.513,149.010 +/- 15.381,278.228 +/- 97.144,42.075 +/- 9.081
1,,,,,,,
2,9_nell_sports_nell_finances*,161.335 +/- 80.124,122.907 +/- 24.199,151.915 +/- 42.799,165.484 +/- 30.724,278.228 +/- 97.144,42.075 +/- 9.081


,Experiment,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD,TreeBoostler,RDN-B
0,10_nell_finances_nell_sports,445.601 +/- 240.560,485.648 +/- 59.599,647.961 +/- 153.561,640.875 +/- 149.893,349.809 +/- 11.872,None
1,,,,,,,
2,10_nell_finances_nell_sports*,367.649 +/- 182.369,422.773 +/- 146.837,581.282 +/- 89.741,437.235 +/- 96.058,349.809 +/- 11.872,None
